In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
torch.manual_seed(42)

In [3]:
df = pd.read_csv('D:/github/pytorch_/fashion-mnist_train.csv/fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
device = torch.device('cuda')
print(f'Using device: {device}')

Using device: cuda


In [5]:
# train test split

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# scaling the feautures
X_train = X_train/255.0
X_test = X_test/255.0

In [8]:
# create CustomDataset Class
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):

    return len(self.features)

  def __getitem__(self, index):

    return self.features[index], self.labels[index]

In [9]:
# create train_dataset object
train_dataset = CustomDataset(X_train, y_train)

In [10]:
# create test_dataset object
test_dataset = CustomDataset(X_test, y_test)

In [11]:
# create train and test loader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, pin_memory = True)

In [15]:
class MyNN(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.BatchNorm1d(128), # batch normalization layer
            nn.ReLU(),
            nn.Dropout(p=0.3), # added dropout layer
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3), # added dropout layer
            nn.Linear(64, 10) # don't need to add softmax layer
        )

    def forward(self, x):
        return self.model(x)


In [16]:
epochs = 100
learning_rate = 0.1

In [17]:
model = MyNN(X_train.shape[1])

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr = learning_rate, weight_decay=1e-4)

In [18]:
for epoch in range(epochs):

    total_epoch_loss = 0
    
    for batch_features, batch_labels in train_loader:

        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)

        loss = criterion(outputs, batch_labels)

        #back pass
        
        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step() # updating gradient

        total_epoch_loss += loss.item()

    avg_loss = total_epoch_loss/len(train_loader)

    print(f'Epoch: {epoch+1}, Loss: {avg_loss}')
        

Epoch: 1, Loss: 0.6543706568876902
Epoch: 2, Loss: 0.47907072758674624
Epoch: 3, Loss: 0.44143229031562803
Epoch: 4, Loss: 0.4178104122877121
Epoch: 5, Loss: 0.40090451045831044
Epoch: 6, Loss: 0.3859019159475962
Epoch: 7, Loss: 0.3766798526843389
Epoch: 8, Loss: 0.36118669176101686
Epoch: 9, Loss: 0.3572296617825826
Epoch: 10, Loss: 0.3456229096651077
Epoch: 11, Loss: 0.33996475001176196
Epoch: 12, Loss: 0.3339437750975291
Epoch: 13, Loss: 0.3264454201459885
Epoch: 14, Loss: 0.31833543320496877
Epoch: 15, Loss: 0.31801520522435506
Epoch: 16, Loss: 0.30921511820952097
Epoch: 17, Loss: 0.3034545786778132
Epoch: 18, Loss: 0.29863241509596505
Epoch: 19, Loss: 0.29961873122056326
Epoch: 20, Loss: 0.2922446951071421
Epoch: 21, Loss: 0.2891357374191284
Epoch: 22, Loss: 0.28644716274738313
Epoch: 23, Loss: 0.28097552438577017
Epoch: 24, Loss: 0.2794969607591629
Epoch: 25, Loss: 0.27698750019073487
Epoch: 26, Loss: 0.2703607622385025
Epoch: 27, Loss: 0.26690397719542186
Epoch: 28, Loss: 0.2668

In [19]:
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [20]:
total = 0
correct = 0

with torch.no_grad():

    for batch_features, batch_labels in test_loader:

        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)

        _ , predicted = torch.max(outputs, 1)

        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

print(correct/total)

0.877


In [21]:
total = 0
correct = 0

with torch.no_grad():

    for batch_features, batch_labels in train_loader:

        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features)

        _ , predicted = torch.max(outputs, 1)

        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()

print(correct/total)

0.9436666666666667
